In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin
/kaggle/input/datadata/DepressionDset.csv
/kaggle/input/additional/new_depressed.csv


In [70]:
data = pd.read_csv('/kaggle/input/additional/new_depressed.csv')
data.head()

,Unnamed: 0,text,Label,cleaned_tweets
0,0,@crimson_angel99 Thank you so much sweetie! 😊❤️,depressed,crimson_angel99 thank much sweetie
1,1,I am who I am. I like what I like. I love what...,depressed,like like love love want get back deal life
2,2,#GrowingUpWithAnxiety people thinking you're r...,depressed,#growingupwithanxiety people thinking youre ru...
3,3,"Don’t talk, just act. Don’t say, just show. Do...",depressed,dont talk act dont say show dont promise prove
4,4,RT @localish: Claudia Martinez underwent SIX b...,not_depressed,localish claudia martinez underwent six brain ...


In [71]:
data.columns

Index(['Unnamed: 0', 'text', 'Label', 'cleaned_tweets'], dtype='object')

In [72]:
data = data.drop(columns=['Unnamed: 0', 'text'])
data.head()

,Label,cleaned_tweets
0,depressed,crimson_angel99 thank much sweetie
1,depressed,like like love love want get back deal life
2,depressed,#growingupwithanxiety people thinking youre ru...
3,depressed,dont talk act dont say show dont promise prove
4,not_depressed,localish claudia martinez underwent six brain ...


In [73]:
import logging
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec

In [74]:
data.head()

,Label,cleaned_tweets
0,depressed,crimson_angel99 thank much sweetie
1,depressed,like like love love want get back deal life
2,depressed,#growingupwithanxiety people thinking youre ru...
3,depressed,dont talk act dont say show dont promise prove
4,not_depressed,localish claudia martinez underwent six brain ...


In [75]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin', binary=True)
word_vectors.init_sims(replace=True)

In [76]:
#word averaging
def word_averaging(wv, words):
    all_words, mean = set(), []
    
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.vocab:
            mean.append(wv.syn0norm[wv.vocab[word].index])
            all_words.add(wv.vocab[word].index)

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        
        return np.zeros(wv.vector_size,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, post) for post in text_list ])


In [77]:
def w2v_tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [78]:
data['cleaned_tweets'] = data['cleaned_tweets'].replace("#", "").replace("_", " ")
data['cleaned_tweets'] = data['cleaned_tweets'].astype(str)


In [79]:
train, test = train_test_split(data, test_size = 0.3, random_state = 42)
test_tokenized = test.apply(lambda r: w2v_tokenize_text(r['cleaned_tweets']), axis=1).values
train_tokenized = train.apply(lambda r: w2v_tokenize_text(r['cleaned_tweets']), axis=1).values

In [80]:
X_train_word_average = word_averaging_list(word_vectors, train_tokenized)
X_test_word_averaging = word_averaging_list(word_vectors, test_tokenized)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `syn0norm` (Attribute will be removed in 4.0.0, use self.vectors_norm instead).
  if __name__ == '__main__':


In [83]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(n_jobs = 1, C = 1e5)
lr = lr.fit(X_train_word_average, train['Label'])
preds = lr.predict(X_test_word_averaging)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [85]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print('accuracy %s' % accuracy_score(preds, test.Label))
print(classification_report(test.Label, preds))

accuracy 0.933055733504164


NameError: name 'classification_report' is not defined